<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-emergent/blob/main/EMERGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EMERGENT: barcodE iMage EnhanceR usinG dEep Neural neTworks

Autoři: Alexej Gaj, Štěpán Studenovský

---
Cílem tohoto projektu bylo vytvořit program na rozpoznávání a čtení čárových EAN a QR kódů, který by byl integrovatelný do zařízení používající fotoaparát. Volili jsme dva přístupy řešení, první cesta byla jít přes:

> detekce kódu -> vyříznutí -> denoising -> čtení

> denoising -> detekce -> vyříznutí -> čtení

K detekci kódu jsme se snažili použít nejprve vlastní síť vytvořenou from scratch se špatnými výsledky, tak jsme zvolili finetunování modelu Yolov5s/m. Vyříznutí jsme napsali simple python skript, který vezme výsledky z yolov5 a vyřízne obrázek.

... detekce - todo

Čtení kódu jsme použili již implementovanou čtečku kódu viz. odkaz???

# Metodologie

Pro detekci čárových kódu jsme nejprve připravili datasety, některé již anotované: https://www.kaggle.com/datasets/whoosis/barcode-detection-annotated-dataset, http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html,

dále ještě jeden ze stránky kaggle.com, kde bylo přibližně 500 obrázků anotovaných ručně pomocí https://www.makesense.ai/, výborný nastroj pro anotování obrázků pro formát YOLO. Obrázky jsme transformovali

Co se týče YOLO, jelikož se jedná o poměrně jednoduchou úlohu detekce jen jedné třídy, tak jsme nejprve zvolili cestu pro vytvoření a natrénování vlastní neuronové síte v jazyce pytorch-lightning. Trénovali jsme na Helios na FJFI, dále v google.comab na T4.